In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Read the CSV and Perform Basic Data Cleaning

In [5]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [6]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Split the Data into Training and Testing

In [7]:
print(df.keys())

Index(['loan_amnt', 'int_rate', 'installment', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'dti',
       'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal',
       'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_amnt', 'next_pymnt_d', 'collections_12_mths_ex_med',
       'policy_code', 'application_type', 'acc_now_delinq', 'tot_coll_amt',
       'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m',
       'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util',
       'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util',
       'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_si

In [8]:
# # Create our features
df2 = df.drop(['home_ownership','issue_d','verification_status', 'pymnt_plan'], axis=1)
df2.head()

X = pd.get_dummies(df2)

# # df(['loan_amnt', 'int_rate', 'installment','annual_inc','dti','delinq_2yrs'	inq_last_6mths	open_acc	pub_rec	revol_bal	...	issue_d_Mar-2019	pymnt_plan_n	initial_list_status_f	initial_list_status_w	next_pymnt_d_Apr-2019	next_pymnt_d_May-2019	application_type_Individual	application_type_Joint App	hardship_flag_N	debt_settlement_flag_N'])
# # df2= df(['loan_amnt','int_rate','installment','annual_inc','dti','delinq_2yrs','inq_last_6mths','open_acc','pub_rec','revol_bal','...','issue_d_Mar-2019','pymnt_plan_n','initial_list_status_f','initial_list_status_w','next_pymnt_d_Apr-2019','next_pymnt_d_May-2019','application_type_Individual','application_type_Joint App','hardship_flag_N','debt_settlement_flag_N',
# # ])
# # df2.head()

# # df2 = df.copy(column=['loan_amnt','int_rate','installment','annual_inc','dti','delinq_2yrs','inq_last_6mths','open_acc','pub_rec','revol_bal','...','issue_d_Mar-2019','pymnt_plan_n','initial_list_status_f','initial_list_status_w','next_pymnt_d_Apr-2019','next_pymnt_d_May-2019','application_type_Individual','application_type_Joint App','hardship_flag_N','debt_settlement_flag_N')
# # df2= df.loc[:,['loan_amnt','int_rate','installment','annual_inc','dti','delinq_2yrs','inq_last_6mths','open_acc','pub_rec','revol_bal','issue_d_Mar-2019','pymnt_plan_n','initial_list_status_f','initial_list_status_w','next_pymnt_d_Apr-2019','next_pymnt_d_May-2019','application_type_Individual','application_type_Joint App','hardship_flag_N','debt_settlement_flag_N']].copy()
#                       # Create our target
y = df2['loan_status']
print(y)

0        low_risk
1        low_risk
2        low_risk
3        low_risk
4        low_risk
           ...   
68812    low_risk
68813    low_risk
68814    low_risk
68815    low_risk
68816    low_risk
Name: loan_status, Length: 68817, dtype: object


In [9]:
# # Define the features set.
# X = df.copy()
# X = X.drop("loan_status", axis=1)
# X.head()

# # Define the target set.
# y = df["loan_status"]
# y[:5]

In [10]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,loan_status_high_risk,loan_status_low_risk,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.005042,0.994958,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.070831,0.070831,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [11]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [12]:
from sklearn.model_selection import train_test_split
# YOUR CODE HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state =42)

In [13]:
# # Creating a StandardScaler instance.
# scaler = StandardScaler()
# # Fitting the Standard Scaler with the training data.
# X_scaler = scaler.fit(X_train)

# # Scaling the data.
# X_train_scaled = X_scaler.transform(X_train)
# X_test_scaled = X_scaler.transform(X_test)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [14]:
# Resample the training data with the BalancedRandomForestClassifier
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78) 

In [15]:
# Calculated the balanced accuracy score
# Fitting the model
rf_model = rf_model.fit(X_train, y_train)
predictions = rf_model.predict(X_test)
predictions

array(['low_risk', 'low_risk', 'low_risk', ..., 'low_risk', 'low_risk',
       'low_risk'], dtype=object)

In [16]:
# Display the confusion matrix
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,88,0
Actual 1,0,13676


In [17]:
# Print the imbalanced classification report
# YOUR CODE HERE
# acc_score = accuracy_score(y_test, predictions)
print("Confusion Matrix")
display(cm_df)
# print(f"Accuracy Score : {acc_score}")
# print("Classification Report")
# print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,88,0
Actual 1,0,13676


In [18]:
# List the features sorted in descending order by feature importance
importances = rf_model.feature_importances_
importances

array([1.53329201e-03, 1.02949027e-03, 1.33054094e-03, 1.30381781e-03,
       1.69305025e-03, 2.27529366e-04, 3.95930226e-04, 6.74143785e-04,
       7.35181052e-05, 1.52975001e-03, 1.16421090e-03, 2.02492570e-03,
       1.97022029e-03, 3.53069013e-02, 2.86679163e-02, 3.87469492e-02,
       4.17162630e-02, 3.00404785e-03, 0.00000000e+00, 0.00000000e+00,
       3.00491105e-02, 1.20733202e-04, 0.00000000e+00, 0.00000000e+00,
       7.71442154e-04, 1.17803313e-03, 5.98445000e-04, 4.93324055e-04,
       5.64521228e-04, 1.21160576e-03, 1.37658010e-03, 1.46584899e-03,
       1.18175709e-03, 6.58102083e-04, 5.22386960e-04, 1.93191420e-03,
       1.33894802e-03, 1.79814728e-03, 8.45644420e-04, 6.12632898e-04,
       1.14583912e-03, 1.00222675e-03, 1.53655218e-03, 1.84313404e-03,
       1.27793366e-03, 0.00000000e+00, 0.00000000e+00, 1.37982353e-03,
       1.71813011e-03, 6.99944913e-04, 7.07146371e-04, 4.59533435e-04,
       1.02912856e-03, 1.42324776e-03, 3.29797406e-04, 8.77809007e-04,
      

In [19]:
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)


[(0.3779400806748278, 'loan_status_high_risk'),
 (0.3776343512033564, 'loan_status_low_risk'),
 (0.04171626302621526, 'total_rec_int'),
 (0.038746949154775286, 'total_rec_prncp'),
 (0.03530690129236802, 'total_pymnt'),
 (0.03004911047743903, 'last_pymnt_amnt'),
 (0.028667916348880544, 'total_pymnt_inv'),
 (0.0030040478536963334, 'total_rec_late_fee'),
 (0.0020249257014476785, 'out_prncp'),
 (0.0019702202875816126, 'out_prncp_inv'),
 (0.0019319141999919458, 'max_bal_bc'),
 (0.00187769260494994, 'next_pymnt_d_May-2019'),
 (0.0018431340365855703, 'bc_open_to_buy'),
 (0.0018347216740696377, 'tot_hi_cred_lim'),
 (0.0017981472791714637, 'total_rev_hi_lim'),
 (0.0017181301073775242, 'mo_sin_old_rev_tl_op'),
 (0.001693050249767482, 'dti'),
 (0.0016530452237656192, 'next_pymnt_d_Apr-2019'),
 (0.001608723893022087, 'total_bc_limit'),
 (0.0015606476029383996, 'total_bal_ex_mort'),
 (0.0015365521797925368, 'avg_cur_bal'),
 (0.0015332920149172167, 'loan_amnt'),
 (0.0015297500131816767, 'revol_bal')

### Easy Ensemble AdaBoost Classifier

In [20]:
# Train the EasyEnsembleClassifier
X = df2.copy()
X = X.drop("loan_status", axis=1)
y = df["loan_status"].values
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X_train, X_test, y_train, y_test = train_test_split(X,
   y, random_state=1)

In [21]:
from sklearn.ensemble import GradientBoostingClassifier
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
   classifier = GradientBoostingClassifier(n_estimators=20,
   learning_rate=learning_rate,
   max_features=5,
   max_depth=3,
   random_state=0)
   classifier.fit(X_train, y_train.ravel)

ValueError: could not convert string to float: 'f'

In [22]:
# Calculated the balanced accuracy score
print("Learning rate: ", learning_rate)
print("Accuracy score (training): {0:.3f}".format(
       classifier.score(
           X_train,
           y_train)))
print("Accuracy score (validation): {0:.3f}".format(
       classifier.score(
           X_test,
           y_test)))
    
classifier = GradientBoostingClassifier(n_estimators=20,
   learning_rate=0.5, max_features=5, max_depth=3, random_state=0)

classifier.fit(X_train, y_train)
predictions = classifier.predict(X_test_scaled)

Learning rate:  0.05


ValueError: could not convert string to float: 'f'

In [23]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
   cm, index=["Actual 0", "Actual 1"],
   columns=["Predicted 0", "Predicted 1"]
)
display(cm_df)

ValueError: Found input variables with inconsistent numbers of samples: [17205, 13764]

In [24]:
# Print the imbalanced classification report
print("Classification Report")
print((y_test, predictions))

Classification Report
(array(['low_risk', 'low_risk', 'low_risk', ..., 'low_risk', 'low_risk',
       'low_risk'], dtype=object), array(['low_risk', 'low_risk', 'low_risk', ..., 'low_risk', 'low_risk',
       'low_risk'], dtype=object))
